In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
from langchain_gigachat.chat_models import GigaChat

In [3]:
llm = GigaChat(
    model="GigaChat-2-Max",
    verify_ssl_certs=False,
    profanity_check=False
)

llm.invoke("hello")

AIMessage(content='Привет!', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 1, 'completion_tokens': 5, 'total_tokens': 6, 'precached_prompt_tokens': 3816}, 'model_name': 'GigaChat-2-Max:2.0.29.04', 'x_headers': {'x-request-id': '3dfb0b8b-2970-4a98-b27b-a31b59a1f6c8', 'x-session-id': 'ee380827-cc08-4c29-9316-7661272562b4', 'x-client-id': None}, 'finish_reason': 'stop'}, id='3dfb0b8b-2970-4a98-b27b-a31b59a1f6c8', usage_metadata={'output_tokens': 5, 'input_tokens': 1, 'total_tokens': 6, 'input_token_details': {'cache_read': 3816}})

In [4]:
from pydantic import BaseModel, Field


class Question(BaseModel):
    """Информация о запросе"""

    question: str = Field(..., description="запрос, который ты создал на основе описания")


class Queries(BaseModel):
    """Информация о всех запросах, которые ты придумал для данного пользователя по данной теме"""

    queries: list[Question]

In [5]:
from langchain_core.prompts import ChatPromptTemplate

template = """Ты — эксперт по созданию реалистичных пользовательских запросов для тестирования ИИ-ассистентов.
Твоя задача — сгенерировать 3 варианта различных запросов для подачи их в агента, занимающегося ресёрчем, соответствующих следующим параметрам:

1. **Профиль пользователя**: {persona} 
2. **Тема**: {topic}
3. **Тип запроса**: {scenario}

**Инструкции**:
- Учитывай особенности профиля
- Адаптируй сложность и тон запроса под тематику.  
- Следуй указанному типу запроса
- Избегай общих фраз — делай запросы конкретными."""

prompt = ChatPromptTemplate.from_template(template)

In [6]:
structured_llm = llm.with_structured_output(Queries)

In [7]:
chain = prompt | structured_llm 

In [10]:
import json
import time
from datetime import datetime
from itertools import product
from tenacity import retry, stop_after_attempt, wait_exponential
from gigachat import GigaChat
from rich.progress import track
from rich import print


personas = ["Занятый специалист. Он явно торопится, используя сокращенный язык и минимум подробностей.",
            "Начинающий. Он студент или новичок в теме, нуждается в объяснении «на пальцах».",
            "Критик. Он любит заковыристые вопросы, придирается ко всему.", 
            "Любопытный. Он любит абстрактные или гипотетические вопросы."]
topics = ["LLM", 
          "Кибербезопасность", 
          "Экономика", 
          "Социология"]
scenarios = ["Чёткий узконаправленный запрос",
             "Неоднозначный вопрос",
             "Многослойный вопрос",
             "Запрос с ошибкой",
             "Запрос на сравнение"]


@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1))
def generate_queries(persona, topic, scenario):
    try:
        queries = chain.invoke({"persona": persona, "topic": topic, "scenario": scenario})
        result = [queries.queries[i].question for i in range(len(queries.queries))]
        return {"queries": result}
    except Exception as e:
        print(f"[red]Ошибка парсинга для: {persona}, {topic}, {scenario}[/red]")
        print(e)
        return {"queries": []}

dataset = []
combinations = list(product(personas, topics, scenarios))

print(f"[bold green]Начата генерация {len(combinations)} комбинаций...[/bold green]")

for persona, topic, scenario in track(combinations, description="Генерация запросов..."):
    result = generate_queries(persona, topic, scenario)
    
    for query in result['queries']:
        dataset.append({
            "persona": persona,
            "topic": topic,
            "scenario": scenario,
            "query": query,
            "meta": {
                "generation_time": datetime.now().isoformat(),
            }
        })
    
    time.sleep(1) 

with open('dataset.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False, indent=2)

print(f"[bold green]Готово! Сгенерировано [cyan]{len(dataset)}[/cyan] запросов.[/bold green]")

Начата генерация 80 комбинаций...

Output()

Готово! Сгенерировано 240 запросов.